In [1]:
import rasterio
import numpy as np
from affine import Affine


In [2]:
fb_path = 'data/humdata/population_nga_2018-10-01.tif'
grid_path = 'data/grid3/NGA - population - v1.2 - mastergrid.tif'

fb_raster = rasterio.open(fb_path)
grid_raster = rasterio.open(grid_path)

In [3]:
# https://stackoverflow.com/questions/639695/how-to-convert-latitude-or-longitude-to-meters
# https://en.wikipedia.org/wiki/Haversine_formula
# earth radius from: https://nssdc.gsfc.nasa.gov/planetary/factsheet/earthfact.html

def coord_distance_to_meter_distance(coord1, coord2, decimals=-1):
    """Use haversine formula to compute distance in meters between two points of coordinates."""
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    R = 6371 # earth radius in km
    dLat = lat2 * np.math.pi / 180 - lat1 * np.math.pi / 180
    dLon = lon2 * np.math.pi / 180 - lon1 * np.math.pi / 180
    a = np.math.sin(dLat/2) * np.math.sin(dLat/2) +\
    np.math.cos(lat1 * np.math.pi / 180) * np.math.cos(lat2 * np.math.pi / 180) *\
    np.math.sin(dLon/2) * np.math.sin(dLon/2)
    c = 2 * np.math.atan2(np.math.sqrt(a), np.math.sqrt(1-a))
    d = R * c
    if decimals < 0:
        return d * 1000 # meters
    else:
        return np.around(d * 1000, decimals)

In [4]:
print(f'fb_raster.transform:\n{fb_raster.transform}\n')
print(f'fb_raster.transform.c:\n{fb_raster.transform.c}\n')
print(f'fb_raster.transform.f:\n{fb_raster.transform.f}\n')

print(f'(1) fb_raster.transform * (0,0):\n\t{fb_raster.transform * (0,0)}\n')
print(f'(2) fb_raster.xy(0,0, offset="ul"):\n\t{fb_raster.xy(0,0,offset="ul")}\n')
print(f'(3) fb_raster.xy(0,0) (center of pixel by default):\n\t{fb_raster.xy(0,0)}\n')

print(f'dist(1,2) = {coord_distance_to_meter_distance(fb_raster.transform * (0,0), fb_raster.xy(0,0,offset="ul"), decimals=3)} meters (rounded to 3 decimals)')
print(f'dist(1,3) = {coord_distance_to_meter_distance(fb_raster.transform * (0,0), fb_raster.xy(0,0), decimals=3)} meters (rounded to 3 decimals)')

# so need to get upper-left corner of pixel for transform!
# apparently, tranform maps to the upper-left corner of the first pixel, meaning the one at (0,0)

fb_raster.transform:
| 0.00, 0.00, 2.68|
| 0.00,-0.00, 13.89|
| 0.00, 0.00, 1.00|

fb_raster.transform.c:
2.682916666666667

fb_raster.transform.f:
13.889027777777779

(1) fb_raster.transform * (0,0):
	(2.682916666666667, 13.889027777777779)

(2) fb_raster.xy(0,0, offset="ul"):
	(2.682916666666667, 13.889027777777779)

(3) fb_raster.xy(0,0) (center of pixel by default):
	(2.6830555555555557, 13.88888888888889)

dist(1,2) = 0.0 meters (rounded to 3 decimals)
dist(1,3) = 21.829 meters (rounded to 3 decimals)


In [5]:
def make_bounds(lat, lon, spread):
    return {
        'left': lon - spread,
        'bottom': lat - spread,
        'right': lon + spread,
        'top': lat + spread
    }

In [6]:
# https://rasterio.readthedocs.io/en/latest/api/rasterio.io.html?highlight=datasetreader.read#rasterio.io.DatasetReader.read
# for windows derived from geo-cooordinates, DatasetReader.read does resampling, making it harder to
# retrieve the exact spatial mapping, therefore use this method instead

def get_window(raster, lat, lon, spread):
    """
    For a box with center (lat,lon) and given spread, get pixel that contains the upper-left corner and
    the pixel that contains the lower-right corner. With indices of those two pixels, define a window as
    ((from_row, to_row),(from_column, to_column)).
    Also return window_anchor_ul: the geo-coordinates of the upper-left corner of the first
    (uppermost-leftmost) pixel of the window.

    :param raster: rasterio.io.DatasetReader
    :param lat: float
    :param lon: float
    :param spread: float
    :return: ((int,int),(int,int)), (float,float)
    """
    bounds = make_bounds(lat, lon, spread)
    row_upper, col_left = raster.index(bounds['left'], bounds['top'], precision=15)
    row_lower, col_right = raster.index(bounds['right'], bounds['bottom'], precision=15)
    window = ((row_upper, row_lower), (col_left, col_right))
    window_anchor_ul = raster.xy(row_upper, col_left, offset="ul")
    return window, window_anchor_ul


In [7]:
lat, lon = (6.541456, 3.312719) # Lagos, Nigeria
spread = 0.1

fb_window, fb_anchor = get_window(fb_raster, lat, lon, spread)
grid_window, grid_anchor = get_window(grid_raster, lat, lon, spread)

In [8]:
lagos_fb_data = fb_raster.read(1,window=fb_window)
lagos_grid_data = grid_raster.read(1, window=grid_window)

In [12]:
transform = Affine(fb_raster.transform.a, fb_raster.transform.b, fb_anchor[0],
                   fb_raster.transform.d, fb_raster.transform.e, fb_anchor[1])

with rasterio.open(\
    'data/out/fb-lagos-0.5.tif', \
    'w', \
    driver='GTiff', \
    width=lagos_fb_data.shape[1], \
    height=lagos_fb_data.shape[0], \
    count=1, \
    dtype=lagos_fb_data.dtype, \
    crs=fb_raster.crs, \
    transform=transform \
    ) as dst:
    dst.write(lagos_fb_data, indexes=1)


In [13]:
fb_lagos_raster = rasterio.open('data/out/fb-lagos-0.5.tif')

In [15]:
# sanity check

print(f'fb_raster.xy(row_upper, col_left):\n\t{fb_raster.xy(fb_window[0][0]+100, fb_window[1][0]+10)}\n')
print(f'fb_lagos_raster.xy(0,0):\n\t{fb_lagos_raster.xy(100,10)}\n')
print(f'dist = {coord_distance_to_meter_distance(fb_raster.xy(fb_window[0][0]+100, fb_window[1][0]+10), fb_lagos_raster.xy(100,10), decimals=3)} meters (rounded to 3 decimals)')

fb_raster.xy(row_upper, col_left):
	(3.2155555555555555, 6.613611111111111)

fb_lagos_raster.xy(0,0):
	(3.2155555555555555, 6.613611111111113)

dist = 0.0 meters (rounded to 3 decimals)


In [16]:
# https://stackoverflow.com/questions/1208118/using-numpy-to-build-an-array-of-all-combinations-of-two-arrays
fb_pixels = np.array(np.meshgrid(
    np.arange(fb_window[0][0], fb_window[0][1]),
    np.arange(fb_window[1][0], fb_window[1][1]))).T.reshape(-1,2)

In [17]:
def process_data(data):
    data = np.nan_to_num(data) # replace nan with zero
    data[data > 0] = 1
    return data.astype(np.uint8)

lagos_fb_data = process_data(lagos_fb_data)

In [19]:
# for each pixel in fb, get coordinates
fb_vxy = np.vectorize(fb_raster.xy) # gets center
fb_xcoords, fb_ycoords = fb_vxy(fb_pixels[:,0], fb_pixels[:,1])

In [20]:
# get corresponding pixels in grid3
grid_vindex = np.vectorize(grid_raster.index)
grid_pixels = np.vstack(grid_vindex(fb_xcoords, fb_ycoords, op=round, precision=15)).T

In [21]:
grid_pixel_counts = np.unique(grid_pixels, return_counts=True, axis=0)

In [22]:
# https://www.geeksforgeeks.org/differences-flatten-ravel-numpy/

# TODO
grid_pixel_unique, convolution = np.unique(grid_pixels[lagos_fb_data.ravel() > 0], return_counts=True, axis=0)

In [23]:
threshold = 3
threshold_mask = convolution >= threshold
convolution[threshold_mask] = 1
convolution[np.invert(threshold_mask)] = 0

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
from sklearn.metrics import confusion_matrix
confusion_matrix(, convolution)

### NOTE fb data for all Nigeria has 1491937976 entries, which if stored as np.uint8, are ca. 1,5 GB
#convolution.nbytes

In [1]:
class ArrayIterator:

    def __init__(self, raster, window_height, window_width):
        self.raster = raster
        self.window_height = window_height
        self.window_width = window_width
        self.current_window = ((0,window_height),(0,window_width))
        self.reached_end = False

    def go_to_next(self):
        # if not yet reached end of row
        if self.current_window[1][1]  < self.raster.width:
            self.current_window = (\
                self.current_window[0],\
                (self.current_window[1][1], self.current_window[1][1] + self.window_width)\
            )
        # if reached end of the row, but not end of table
        elif self.current_window[0][1] < self.raster.height:
            self.current_window = (\
                (self.current_window[0][1], self.current_window[0][1] + self.window_height),\
                (0, self.window_width)\
            )
        # if reached end of table
        else:
            self.reached_end = True
            #raise IndexError("Reached end of table; no next window.")

    def pop_window(self):
        current_window = self.current_window
        self.go_to_next()
        return current_window

    def has_reached_end(self):
        return self.reached_end

    def reset(self):
        self.current_window = ((0,window_height),(0,window_width))
        self.reached_end = False

In [2]:
def prepare_data(data):
    data = np.nan_to_num(data) # replace nan with zero
    data[data > 0] = 1 # make binary
    return data.astype(np.uint8)

# https://stackoverflow.com/questions/1208118/using-numpy-to-build-an-array-of-all-combinations-of-two-arrays
def get_pixels(window):
    return np.array(np.meshgrid(
        np.arange(window[0][0], window[0][1]),
        np.arange(window[1][0], window[1][1]))).T.reshape(-1,2)

In [3]:
import rasterio
import numpy as np

In [4]:
# define paths and open rasters
fb_path = 'data/humdata/population_nga_2018-10-01.tif'
grid_path = 'data/grid3/NGA - population - v1.2 - mastergrid.tif'

fb_raster = rasterio.open(fb_path)
grid_raster = rasterio.open(grid_path)

In [5]:
# find appropriate window shape for fb raster
# prime factorization of fb_raster.height=34558: 2 * 37 * 467
# prime factorization of fb_raster.width=43172: 2 * 2 * 43 * 251
# choose window shape: (467,251)

In [6]:
# initialize zeros array of same shape as GRID raster

result = np.zeros(grid_raster.shape, dtype=np.uint8)

In [7]:
# initialize iterator
window_iterator = ArrayIterator(fb_raster, 467, 251)

In [8]:
while not window_iterator.has_reached_end():
    window = window_iterator.pop_window()
    assert (window[0][1]-window[0][0]) == 467
    assert (window[1][1]-window[1][0]) == 251
    # read data from FB raster using current window
    data = fb_raster.read(1,window=window)
    assert np.all(data.shape == (467,251))
    # replace nan with 0 and >0 with 1
    data = prepare_data(data)
    # get all pixels contained in window
    pixels = get_pixels(window)
    # keep only those pixels for which data has a 1 entry
    pixels = pixels[data.ravel() > 0]
    # check if there are any pixels, if continue from next iteration
    if pixels.size > 0:
        # use FB raster to get coordinates for each pixel
        fb_raster_vxy = np.vectorize(fb_raster.xy) # gets center
        xcoords, ycoords = fb_raster_vxy(pixels[:,0], pixels[:,1])
        # for each coordinate get corresponding pixel in the GRID raster
        grid_raster_vindex = np.vectorize(grid_raster.index)
        grid_pixels = np.vstack(grid_raster_vindex(xcoords, ycoords, op=round, precision=15)).T
        # get unique counts for pixels in GRID raster
        grid_pixels_unique, counts = np.unique(grid_pixels, return_counts=True, axis=0)
        # update result
        result[grid_pixels_unique[:,0], grid_pixels_unique[:,1]] += counts.astype(np.uint8)

# took 17 mins to run

In [10]:
result.tofile('nigeria-fb_to_grid_mapping.csv', ',')

In [ ]:
myresult = np.genfromtxt('nigeria-fb_to_grid_mapping.csv', delimiter=',')